# QNN to learn to determine the correct angle for Rx Gate for performing a "NOT" Gate

In [1]:
import classiq
classiq.authenticate()

/home/devilkillerag/miniconda3/envs/mnistenv/lib/python3.11/site-packages/classiq/_internals/authentication/token_manager.py:82: UserWarning: Device is already registered.
Generating a new refresh token should only be done if the current refresh token is compromised.
To do so, set the overwrite parameter to true
  warnings.warn(


In [17]:
from typing import Dict

from classiq import Model ,synthesize
from classiq.builtin_functions import HardwareEfficientAnsatz
from classiq import QReg
from classiq.applications.qnn import QLayer
from classiq.applications.qnn.datasets import DATALOADER_NOT

import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import DataLoader

from classiq.applications.qnn.types import (
    MultipleArguments,
    SavedResult,
    ResultsCollection
)
from classiq.execution import execute_qnn
from classiq.synthesis import SerializedQuantumProgram

### General Flow

**Step 1: Define Quantum Layer**

Step 1.1: Defining the quantum model and synthesizing it to a quantum circuit

Step 1.2: Defining the execute and post-process callables

Step 1.3: Defining a torch.nn.Module network

**Step 2: Initialise Dataset, Loss Function, and Optimiser**

**Step 3: Learning Process**

**Step 4: Test the QNN**

#### Step 1.1: Create Parametric Quantum Circuit (PQC)

In [3]:
_NUM_QUBITS = 1
_REPS = 1
_CONNECTIVITY_MAP = "circular"

In [4]:
def add_rx(md: Model, prefix: str, in_wire=None) -> Dict[str, QReg]:
    if in_wire is not None:
        kwargs = { "in_wires": { "IN": in_wire["OUT"] } }
    else:
        kwargs = {}

    hwea_params = HardwareEfficientAnsatz(
        num_qubits=_NUM_QUBITS,
        connectivity_map=_CONNECTIVITY_MAP,
        reps=_REPS,
        one_qubit_gates="rx",
        two_qubit_gates=[],
        parameter_prefix=prefix,
    )

    return md.HardwareEfficientAnsatz(hwea_params, **kwargs)

In [5]:
model = Model()
output_1 = add_rx(model, "input_")
output_2 = add_rx(model, "weight_", output_1)

quantum_program = synthesize(model.get_model())

#### Step 1.2: Create the execution and post-processing

In [7]:
def execute(quantum_program: SerializedQuantumProgram, arguments:MultipleArguments) -> ResultsCollection:
    return execute_qnn(quantum_program, arguments) 

In [8]:
# Post-process the result, returning a dict:
# Note: this function assumes that we only care about
#   differentiating a single state (|0>)
#   from all the rest of the states.
#   In case of a different differentiation, this function should change.
def post_process(result: SavedResult) -> torch.Tensor:
    """
    Take in a `SavedResult` with `ExecutionDetails` value type, and return the
    probability of measuring |0> which equals the amount of `|0>` measurements
    divided by the total amount of measurements.
    """
    counts: dict = result.value.counts
    # The probability of measuring |0>
    p_zero: float = counts.get("0", 0.0) / sum(counts.values())
    return torch.tensor(p_zero)

#### Step 1.3: Creating a Network

In [9]:
class QNet(torch.nn.Module):
    def __init__(self, *args, **kwargs) -> None:
        super().__init__()
        self.qlayer = QLayer(
            quantum_program,
            execute,
            post_process,
            *args,
            **kwargs,
        )

    def forward(self, x: torch.Tensor) -> torch.Tensor:
        x = self.qlayer(x)
        return x

In [10]:
model = QNet()

#### Step  2: Choose a Dataset, Loss Function, and Optimizer

In [11]:
_LEARNING_RATE = 1.0

data_loader = DATALOADER_NOT

loss_function = nn.L1Loss()

optimizer = optim.SGD(model.parameters(), lr=_LEARNING_RATE)

#### Step 3: Training

In [12]:
def train(model: nn.Module, data_loader: DataLoader, loss_function: nn.modules.loss._Loss, optimizer: optim.Optimizer, epoch: int = 20) -> None:
    for index in range(epoch):
        print(index, model.qlayer.weight)
        for data, label in data_loader:
            optimizer.zero_grad()
            output = model(data)
            loss = loss_function(output, label)
            loss.backward()
            optimizer.step()

In [13]:
train(model, data_loader, loss_function, optimizer)

0 Parameter containing:
tensor([0.5383], requires_grad=True)
1 Parameter containing:
tensor([0.7946], requires_grad=True)
2 Parameter containing:
tensor([1.1364], requires_grad=True)
3 Parameter containing:
tensor([1.5759], requires_grad=True)
4 Parameter containing:
tensor([2.1130], requires_grad=True)
5 Parameter containing:
tensor([2.6135], requires_grad=True)
6 Parameter containing:
tensor([2.7721], requires_grad=True)
7 Parameter containing:
tensor([2.9553], requires_grad=True)
8 Parameter containing:
tensor([2.9797], requires_grad=True)
9 Parameter containing:
tensor([3.0529], requires_grad=True)
10 Parameter containing:
tensor([3.0773], requires_grad=True)
11 Parameter containing:
tensor([3.1262], requires_grad=True)
12 Parameter containing:
tensor([3.1384], requires_grad=True)
13 Parameter containing:
tensor([3.1384], requires_grad=True)
14 Parameter containing:
tensor([3.1384], requires_grad=True)
15 Parameter containing:
tensor([3.1384], requires_grad=True)
16 Parameter conta

#### Step 4: Testing

In [15]:
def check_accuracy(model: nn.Module, data_loader: DataLoader, atol=1e-4) -> float:
    num_correct = 0
    total = 0
    model.eval()

    with torch.no_grad():
        for data, labels in data_loader:
            predictions = model(data)
            is_prediction_correct = predictions.isclose(labels, atol=atol)
            print(f"data: {data}\n labels: {labels}\n is_prediction_correct: {is_prediction_correct}\n sum: {is_prediction_correct.sum()}\n item: {is_prediction_correct.sum().item()}")
            num_correct += is_prediction_correct.sum().item()
            total += labels.size(0)
    
    accuracy = float(num_correct) / float(total)
    print(f"Test Accuracy of the model: {accuracy*100:.2f}")
    return accuracy


In [16]:
check_accuracy(model, data_loader)

data: tensor([[0.0000],
        [3.1416]])
 labels: tensor([0., 1.])
 is_prediction_correct: tensor([True, True])
 sum: 2
 item: 2
Test Accuracy of the model: 100.00


1.0

The results show that the accuracy is 1, meaning a 100% success rate at performing the required transformation (i.e. the network learned to perform a X-gate). We may further test it by printing the value of model.qlayer.weight, which is a tensor of shape (1,1), which should, after training, be close to pi=3.1416.